In [2]:
import tensorflow as tf

/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
#define weight,bias
def weight_variable(shape):
    init = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)
def bias_variable(shape):
    init = tf.constant(0.1,shape=shape)
    return tf.Variable(init)

In [4]:
#placeholder
xs = tf.placeholder(tf.float32,[None,30,300,1])
ys = tf.placeholder(tf.float32,[None,3])
keep_prob = tf.placeholder(tf.float32)


In [5]:
#define cnn layer
def cov2d(W,x):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding = 'SAME')

In [6]:
#define max-pooling
def  max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [7]:
#conv1 layer
#每一个层都有自己单独的weight和bias 
w_conv1 = weight_variable([5,5,1,32])
bias_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(cov2d(w_conv1,xs)+bias_conv1)
p_conv1 = max_pool_2x2(h_conv1)

In [8]:
#conv2
w_conv2 = weight_variable([5,5,32,64])
bias_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(cov2d(w_conv2,p_conv1)+bias_conv2)
p_conv2 = max_pool_2x2(h_conv2)

In [9]:
#func1
w_fc1 = weight_variable([5*5*64,1024])
b_fc1 = bias_variable([1024])

#flat
h_pool_flat = tf.reshape(p_conv2,[-1,5*5*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,w_fc1)+b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)



In [10]:
#func2
w_fc2 = weight_variable([1024,3])
b_fc2 = bias_variable([3])


prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,w_fc2)+b_fc2)

In [11]:
#define accuracy

def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pred = sess.run(prediction,feed_dict={xs:v_xs,keep_prob:1})
    correct = tf.equal(tf.argmax(y_pred,1),tf.argmax(v_ys,1))
    correct_accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    result = sess.run(correct_accuracy,feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

In [55]:
#cost
cost = tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=ys)


#train
#with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
train_op = tf.train.AdamOptimizer(0.001).minimize(cost)




In [13]:
init = tf.global_variables_initializer()

In [14]:
import pandas as pd

In [15]:
test = pd.read_csv('./data/bert_test.txt',delimiter='\t',names=["label","review"])
val = pd.read_csv('./data/bert_val.txt',delimiter='\t',names=["label","review"])
train = pd.read_csv('./data/bert_train.txt',delimiter='\t',names=["label","review"])


In [16]:
train.shape,val.shape,train.shape

((18000, 2), (9000, 2), (18000, 2))

In [17]:
#load data
x_test = test.review
y_test = test.label
x_train = train.review
y_train = train.label

In [18]:
import jieba

In [19]:
#load dictionary
def loadWord2Vec(filename):
    vocab = []
    embd = []
    cnt = 0
    fr = open(filename,'r')
    line = fr.readline().strip()
    #print line
    word_dim = int(line.split(' ')[1])    
    vocab.append("unk")
    embd.append([0]*word_dim)
    for line in fr :
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print("loaded word2vec") 
    fr.close()
    return vocab,embd


In [20]:
import numpy as np
vocab,embd = loadWord2Vec('/Users/james/Downloads/sgns.zhihu.word')
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

loaded word2vec


In [21]:
def dividesentence(a):
    sentence = []
    seg_list = jieba.cut(a, cut_all=True)
    for item in seg_list:
        sentence.append(item)
    return sentence

In [22]:
def findemdding(str1):
    for i in range(0,vocab_size):
        if vocab[i] == str1:
            return embedding[i]
    else:
        return np.zeros(shape=(300,))

In [23]:
def combinevector(b):
    sentenceembed = []
    if len(b)<=30:
        for i in range(0,len(b)):
            c = findemdding(b[i])
            sentenceembed.append(c)
        return i,np.array(sentenceembed)
    else:
        for i in range(0,30):
            c = findemdding(b[i])
            sentenceembed.append(c)
        return i,np.array(sentenceembed)

In [24]:
def getvect(a):
    b = dividesentence(a)
    count, arr = combinevector(b)
    if count<=30:
        a =  np.zeros(shape=(30-count-1,300))
        d = np.insert(arr ,count+1,values=a,axis=0)
        return d
    else:
        return arr

In [25]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train_one_hot = encoder.fit_transform(y_train)
y_test_one_hot = encoder.fit_transform(y_test)

/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [26]:
np.array(y_train_one_hot).shape

(18000, 3)

In [27]:
#test
x_train1=getvect(x_train[1])
x_train1= np.array(x_train1).reshape([1,30,300,1])
y_train1 = y_train_one_hot[1]
y_train1=np.array(y_train1).reshape([1,3])
x_test1=getvect(x_test[1])
x_test1= np.array(x_test1).reshape([1,30,300,1])
y_test1 = y_test_one_hot[1]
y_test1=np.array(y_test1).reshape([1,3])

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/vr/3b_f9c5j0yvgq7_41pp6_ktm0000gn/T/jieba.cache
Loading model cost 0.884 seconds.
Prefix dict has been built succesfully.


In [28]:
x_train1.shape,y_train1.shape,x_test1.shape,y_test1.shape

((1, 30, 300, 1), (1, 3), (1, 30, 300, 1), (1, 3))

### test data

In [51]:
x_test_vec = []
for i in range(0,3000):
    a = getvect(x_test[i])
    a = np.array(a).reshape([30,300,1])
    x_test_vec.append(a)

In [52]:
x_test_array=np.array(x_test_vec)

In [31]:
y_test_vec = []
for i in range(0,3000):
    a = y_test_one_hot[i]
    y_test_vec.append(a)

In [32]:
y_test_array=np.array(y_test_vec)

#### train data  first 8000


In [33]:
y_train = y_train[:8000]

### test

In [56]:
x_train_vec = []

In [57]:
for i in range(0,2):
    a = getvect(x_train[i])
    b = np.array(a).reshape([30,300,1])
    x_train_vec.append(b)

In [58]:
x_train_array=np.array(x_train_vec)

In [59]:
y_train_vec = []
for i in range(0,2):
    a = y_train_one_hot[i]
    y_train_vec.append(a)

In [60]:
y_train_array=np.array(y_train_vec)

In [38]:
x_train_vec = []
for i in range(0,8000):
    a = getvect(x_train[i])
    b = np.array(a).reshape([1,30,300,1])
    x_train_vec.append(b)

In [35]:
y_train_vec = []
for i in range(0,8000):
    a = y_train_one_hot[i]
    y_train_vec.append(a)

In [36]:
#x_train_array=np.array(x_train_vec)
#y_train_array=np.array(y_train_vec)

### run model

In [37]:
x_test_array.shape,y_test_array.shape

((3000, 30, 300, 1), (3000, 3))

In [38]:
x_train_array.shape, y_train_array.shape

((8000, 30, 300, 1), (8000, 3))

In [44]:
def get_Batch(data, label, batch_size):
    print(data.shape, label.shape)
    input_queue = tf.train.slice_input_producer([data, label], num_epochs=1, shuffle=True, capacity=32 ) 
    x_batch, y_batch = tf.train.batch(input_queue, batch_size=batch_size, num_threads=1, capacity=32, allow_smaller_final_batch=False)
    return x_batch, y_batch


In [1]:
x_batch, y_batch = get_Batch(x_train_array,y_train_array, 1)

NameError: name 'get_Batch' is not defined

In [ ]:
batch_xs,batch_ys = tf.train.batch([x_train_array,y_train_array],batch_size=1)

In [ ]:
batch_xs.shape,batch_ys.shape

In [41]:
np.array(x_train_vec[1]).shape

(1, 30, 300, 1)

In [46]:
np.array(y_train_vec[1]).reshape([1,3])

array([[0, 1, 0]])

In [61]:
#sess.run
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        
        sess.run(train_op,feed_dict={xs:x_train_array,ys:y_train_array,keep_prob:0.5})
        if i % 50 == 0:
            #print(compute_accuracy(x_test_array,y_test_array))

SyntaxError: unexpected EOF while parsing (<ipython-input-61-e113a19e5a58>, line 8)